** We will start by building a sample plugin.  We will need to start with the base class for a Plugin Controller. **

In [1]:
try:
    from src.controller_plugin import ControllerPlugin
except ImportError:
    from sys import path
    path.append("/root/backend-interpreter/plugin_interpreter")
    print(path)
    from src.controller_plugin import ControllerPlugin
    from os import environ
    environ["PORT"] = "10000"                                 #The PCP Framework selects the port you will need to use

['', '/usr/bin', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/lib/python3.6/site-packages', '/usr/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython', '/root/backend-interpreter/plugin_interpreter']


** Filename must be the same as the Class Name.  For this example, we will use filename SamplePlugin.py and Class Name SamplePlugin for the "Sample Plugin" plugin. **
** The __init__ shown below is the minimum requirements for the Plugin.

In [2]:
class SamplePlugin(ControllerPlugin):
    functionality =  []                         # Get the commands you built from the prior tutorial
    def __init__(self):
        """
        The plugin class is the last class to init,
        but Database and logger have not started yet
        Should not be using the database handles or the logger in this fn
        """
        self.name = "SamplePlugin"
        super().__init__(self.name, self.functionality)

** Note that if you instantiate this new SamplePlugin, it will error out since you must write a minimum of a start and a stop function to overwrite the default functions. **

In [3]:
SamplePlugin()

TypeError: Cannot instantiate abstract class SamplePlugin with abstract methods _stop, start

** We will start again with the Plugin Class definition and build the full Plugin. **

In [4]:
class SamplePlugin(ControllerPlugin):
    functionality =  []                         # Get the commands you built from the prior tutorial
    def __init__(self):
        """
        The plugin class is the last class to init,
        but Database and logger have not started yet
        Should not be using the database handles or the logger in this fn
        """
        self.name = "SamplePlugin"
        super().__init__(self.name, self.functionality)
    def start(self, logger, ext_signal):
        """
        Start function is called by the supervisor only
        This function may assume the logger and the db is already start()ed
        This function should include all calls required to run until ext_signal says to stop.

        :param logger: <central logger> object for all log handling
        :param ext_signal: <ctypes.bool> the kill signal
        :return: This function calls exit! when complete
        """
        pass
    def _stop(self):
        pass
    

In [5]:
SamplePlugin()